In [1]:
import nltk
import numpy as np
from nltk.util import ngrams
import glob
import errno
import os
import string
import csv
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from sklearn.model_selection import train_test_split 
from nltk.corpus import wordnet
  
data=[]
N=3
K=1#smoothing
v_occur=1
exclude = set(string.punctuation)
stop_words = set(stopwords.words('english')) 
lemmatizer = WordNetLemmatizer() 


def nltk2wn_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None


def convertTuple(tup): 
    str =  ''.join(tup) 
    return str

with open('Womens Clothing E-Commerce Reviews.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
         data.append(row)
d=np.array(data)
print(len(d))

#separate classes
d=d[1:,1:] #remove first line and column
l_recommendation=np.array(d[:,5])
title=np.array([d[:,2]])
review_text=np.array([d[:,3]])
review=np.concatenate((title.T,review_text.T),axis=1)

X_train, X_test, y_train, y_test = train_test_split(review, l_recommendation, test_size=0.2,random_state=0)
print(len(X_train), len(X_test), len(y_train), len(y_test))
c_p=list(y_train).count('1')
c_n=list(y_train).count('0')
print(list(y_test).count('1'),list(y_test).count('0'))
review_p = X_train[y_train == '1', :] # extract all rows with label 1
review_n = X_train[y_train == '0', :] # extract all rows with label 0
print(len(review_p), len(review_n))


def ngram_text(review,N):
    text=""
    for i in range(len(review)):
        txt=str(review[i])
        s = ''.join(ch for ch in  txt if ch not in exclude)
        out=s.lower()
        result = re.sub(r'\d+', '', out) #remove numbers
        text=text+' '+result
        
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(text))  #tokenizer
    wn_tagged = map(lambda x: (x[0], nltk2wn_tag(x[1])), nltk_tagged) #lemmatizer
    lemmas = []
    for word, tag in wn_tagged:
        if tag is None:            
            lemmas.append(word)
        else:
            lemmas.append(lemmatizer.lemmatize(word, tag))
    filtered_sentence = [w for w in lemmas if not w in stop_words] 
    
    ungs = ngrams(filtered_sentence, 1) #unigram
    ufdist = nltk.FreqDist(ungs)    
    high_fdist = {convertTuple(k) for k, v in ufdist.items() if v >=v_occur} #remove tokens that occurs more than x times
    occurence_sentence = [w for w in filtered_sentence if w in high_fdist] 
    tokens_final=occurence_sentence
    ngs1 = ngrams(tokens_final, N-1)  #n-gram n-1
    nfdist1 = nltk.FreqDist(ngs1)
    ngs = ngrams(tokens_final, N) #Create n-grams
    nfdist = nltk.FreqDist(ngs)    
    print("top 10 words: ", nfdist.most_common(10))
    vocabulary=len(nfdist)    
    prob_uni=dict()
    for k,v in ufdist.items():
        prob_uni[convertTuple(k)]=v
    return tokens_final, ngs, ungs, nfdist1, nfdist, vocabulary, prob_uni

tokens_p, ngs_p, ungs_p, nfdist1_p, nfdist_p, vocabulary_p, prob_uni_p=ngram_text(review_p,N)
print('vocab_p',vocabulary_p)
print('len_tokensp',len(tokens_p))
tokens_n, ngs_n, ungs_n,nfdist1_n, nfdist_n, vocabulary_n, prob_uni_n=ngram_text(review_n,N)
print('vocab_n',vocabulary_n)
print('len_tokens_n',len(tokens_n))
tokens_t, ngs_t, ungs_t, nfdist1_t, nfdist_t, vocabulary_t, prob_uni_t=ngram_text(X_train,N)
print('vocab_t',vocabulary_t)




inference=[]

for i in range(len(X_test)):  
    txt=str(X_test[i])
    s = ''.join(ch for ch in  txt if ch not in exclude) #remove punctuation
    out=s.lower() #lowercase
    result = re.sub(r'\d+', '', out) #remove numbers
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(result))   #tokenizer
    wn_tagged = map(lambda x: (x[0], nltk2wn_tag(x[1])), nltk_tagged) #lemmatizer
    lemmas = []
   
    for word, tag in wn_tagged:
        if tag is None:     
            lemmas.append(word)
        else:
            lemmas.append(lemmatizer.lemmatize(word, tag))
    filtered_sentence = [w for w in lemmas if not w in stop_words] 
    ungs = ngrams(filtered_sentence, 1) #unigram
    ufdist = nltk.FreqDist(ungs)    
    high_fdist = {convertTuple(k) for k, v in ufdist.items() if v >=v_occur} #remove tokens that occurs more than x times
    occurence_sentence = [w for w in filtered_sentence if w in high_fdist] 
    Ntotal=0
    Ptotal=0
    for j in range(len(occurence_sentence)-N+1): 
        Pcondit=0
        Ncondit=0
        if (N==1):
            tuple1=(occurence_sentence[j])
            if tuple1 in tokens_t:
                Pcondit=((prob_uni_p.get(tuple1,0)+K)/(len(tokens_p)+(K*vocabulary_t)))
                Ncondit=((prob_uni_n.get(tuple1,0)+K)/(len(tokens_n)+(K*vocabulary_t)))                
        if (N==2):
            if nfdist_t.get(tuple3,0) >0:
                Pcondit=((nfdist_p.get(tuple3,0)+K)/(prob_uni_p.get(tuple2,0)+(K*vocabulary_t)))
                Ncondit=((nfdist_n.get(tuple3,0)+K)/(prob_uni_n.get(tuple2,0)+(K*vocabulary_t))) 
        if (N==3):
            tuple3a=(occurence_sentence[j],occurence_sentence[j+1],occurence_sentence[j+2])
            tuple3b=(occurence_sentence[j],occurence_sentence[j+1])
            tuple2a=(occurence_sentence[j+1],occurence_sentence[j+2])
            tuple2b=(occurence_sentence[j+1])
            tuple1=(occurence_sentence[j+2])
            if nfdist_t.get(tuple3a,0)>0:
                Pcondit=((nfdist_p.get(tuple3a,0)+K)/(nfdist1_p.get(tuple3b,0)+(K*vocabulary_t)))
                Ncondit=((nfdist_n.get(tuple3a,0)+K)/(nfdist1_n.get(tuple3b,0)+(K*vocabulary_t)))
        if (N==4):
            tuple4a=(occurence_sentence[j],occurence_sentence[j+1],occurence_sentence[j+2],occurence_sentence[j+3])
            tuple4b=(occurence_sentence[j],occurence_sentence[j+1],occurence_sentence[j+2])
            tuple3a=(occurence_sentence[j+1],occurence_sentence[j+2],occurence_sentence[j+3])
            tuple3b=(occurence_sentence[j+1],occurence_sentence[j+2])
            tuple2a=(occurence_sentence[j+2],occurence_sentence[j+3])
            tuple2b=(occurence_sentence[j+2])
            tuple1=(occurence_sentence[j+3])
            if nfdist_t.get(tuple4a,0)>0:
                Pcondit=((nfdist_p.get(tuple4a,0)+K)/(nfdist1_p.get(tuple4b,0)+(K*vocabulary_t)))
                Ncondit=((nfdist_n.get(tuple4a,0)+K)/(nfdist1_n.get(tuple4b,0)+(K*vocabulary_t)))
        if Ncondit!=0:
            Ntotal+=np.log2(Ncondit)
        if Pcondit!=0:
            Ptotal+=np.log2(Pcondit)
    N_fim=np.log2(c_n/(c_p+c_n))+Ntotal
    P_fim=np.log2(c_p/(c_p+c_n))+Ptotal
    
    if(N_fim>P_fim):
        #final="N"
        inference.append('0')
    else:
        #final="P"
        inference.append('1') 


accuracy=0
errors=[]
for i in range(len(inference)):
    if inference[i]==y_test[i]:
                         accuracy+=1
    else:
        errors.append(i)

print(errors)
print(len(errors))
print(accuracy/len(inference))




23487
18788 4698 18788 4698
3870 828
15444 3344
top 10 words:  [(('fit', 'true', 'size'), 353), (('love', 'love', 'love'), 190), (('cant', 'wait', 'wear'), 179), (('run', 'true', 'size'), 152), (('receive', 'many', 'compliment'), 131), (('size', 'fit', 'perfectly'), 115), (('small', 'fit', 'perfectly'), 108), (('fit', 'like', 'glove'), 104), (('get', 'many', 'compliment'), 93), (('usually', 'wear', 'size'), 92)]
vocab_p 395691
len_tokensp 455358
top 10 words:  [(('really', 'want', 'love'), 66), (('want', 'love', 'dress'), 53), (('make', 'look', 'like'), 48), (('really', 'want', 'like'), 44), (('fit', 'true', 'size'), 29), (('want', 'love', 'top'), 27), (('sadly', 'go', 'back'), 26), (('way', 'much', 'fabric'), 22), (('look', 'like', 'maternity'), 22), (('one', 'go', 'back'), 21)]
vocab_n 96709
len_tokens_n 102825
top 10 words:  [(('fit', 'true', 'size'), 382), (('love', 'love', 'love'), 195), (('cant', 'wait', 'wear'), 179), (('run', 'true', 'size'), 161), (('receive', 'many', 'complim